# Optimization Tool for Shipment Quantities

### Problem: We are often asked to quote prices based on which products and how much quantities our customers want. There are many factors to consider before making the decision, but currently we do not have an easy method to solve it.

### This is a tool that helps us to find the optimal quantities of requested items for shipment. Below are what need to be considered:

- The total volume must not exceed a container's capacity (container sizes: 20ft, 40ft, 40hq).  
- There might be some production capacity for certain products, so the shipment quantity cannot go over that.  
- 'Demand' indicates what the customer is requesting for quote. This tool will adjust quantities from this 'Demand' while keeping the constraints and objective.
- The objective is to maximize the profit.

In [2]:
import numpy as np
import pandas as pd
import pulp as pl

#### Below DataFrame contains the volume (CBM), unit profit, and quantity per box (in yards) for each item.

In [3]:
df_box = pd.read_excel("box_sizes.xlsx")
df_box['CBM'] = df_box['Width'] * df_box['Length'] * df_box['Height']
df_box.head()

,Item,Per Box,Width,Length,Height,Unit Profit,CBM
0,Satin Taffeta K-1479,250,1.51,0.21,0.51,0.93,0.161721
1,Taffeta K-2790,250,0.80,0.19,0.33,0.84,0.050160
2,Hi Multi Chiffon K-711,500,1.52,0.23,0.57,0.89,0.199272
3,Rendezvous Chiffon K-2420,500,1.52,0.25,0.61,0.92,0.231800
4,Knit Chiffon K-2108,500,1.52,0.28,0.70,0.60,0.297920


#### Input data: item, demand, and production capacity  
* If there is no capacity limit, put a big number (> 100,000)

In [4]:
import ipywidgets as widgets
from IPython.display import display

# Dropdown for item selection
dropdown = widgets.Dropdown(
    options=df_box['Item'].dropna().unique(),
    description='Select Item:',
    disabled=False,
)

# Input boxes for demand and capacity
demand_box = widgets.IntText(description='Demand:')
capacity_box = widgets.IntText(description='Capacity:')
save_button = widgets.Button(description='Save Data')

# List to store the selections along with demand and capacity
selections = []

# Variable to keep track of the currently selected item
current_item = None

# Function to handle changes in the dropdown
def dropdown_eventhandler(change):
    global current_item
    current_item = change.new

# Function to handle saving the data
def save_data(b):
    if current_item is not None:
        # Check if the item already exists in the selections
        existing_entry = next((entry for entry in selections if entry['item'] == current_item), None)
        
        if existing_entry:
            # Update the existing entry
            existing_entry['demand'] = demand_box.value
            existing_entry['capacity'] = capacity_box.value
        else:
            # Add a new entry
            selections.append({'item': current_item, 'demand': demand_box.value, 'capacity': capacity_box.value})
        
        print(f"Current selections: {selections}")

# Set up event handlers
dropdown.observe(dropdown_eventhandler, names='value')
save_button.on_click(save_data)

# Display the widgets
display(dropdown, demand_box, capacity_box, save_button)

Dropdown(description='Select Item:', options=('Satin Taffeta K-1479 ', 'Taffeta K-2790 ', 'Hi Multi Chiffon K-…

IntText(value=0, description='Demand:')

IntText(value=0, description='Capacity:')

Button(description='Save Data', style=ButtonStyle())

Current selections: [{'item': 'Hi Multi Chiffon K-711 ', 'demand': 50000, 'capacity': 45000}]
Current selections: [{'item': 'Hi Multi Chiffon K-711 ', 'demand': 50000, 'capacity': 45000}, {'item': 'Satin Taffeta K-1479 ', 'demand': 40000, 'capacity': 100000}]
Current selections: [{'item': 'Hi Multi Chiffon K-711 ', 'demand': 50000, 'capacity': 45000}, {'item': 'Satin Taffeta K-1479 ', 'demand': 40000, 'capacity': 100000}, {'item': 'Velvet  (320G)', 'demand': 4000, 'capacity': 100000}]
Current selections: [{'item': 'Hi Multi Chiffon K-711 ', 'demand': 50000, 'capacity': 45000}, {'item': 'Satin Taffeta K-1479 ', 'demand': 40000, 'capacity': 100000}, {'item': 'Velvet  (320G)', 'demand': 4000, 'capacity': 100000}, {'item': 'Arcai Slub K-2574 ', 'demand': 40000, 'capacity': 50000}]
Current selections: [{'item': 'Hi Multi Chiffon K-711 ', 'demand': 50000, 'capacity': 45000}, {'item': 'Satin Taffeta K-1479 ', 'demand': 40000, 'capacity': 100000}, {'item': 'Velvet  (320G)', 'demand': 4000, 'ca

#### Select container size

In [6]:
import ipywidgets as widgets
from IPython.display import display

# Dropdown for container selection
container_dropdown = widgets.Dropdown(
    options=['20ft', '40ft', '40hq'],
    description='Container:',
    disabled=False,
)

# Variable to store the selected container type
selected_container = None

# Function to handle changes in the dropdown
def dropdown_eventhandler(change):
    global selected_container
    selected_container = change.new
    print(f"Selected container: {selected_container}")

# Observe function for the dropdown
container_dropdown.observe(dropdown_eventhandler, names='value')

# Display the dropdown
display(container_dropdown)

Dropdown(description='Container:', options=('20ft', '40ft', '40hq'), value='20ft')

Selected container: 40ft


In [7]:
df_selected = pd.DataFrame(selections)
df_selected

,item,demand,capacity
0,Hi Multi Chiffon K-711,50000,45000
1,Satin Taffeta K-1479,40000,100000
2,Velvet (320G),4000,100000
3,Arcai Slub K-2574,40000,50000
4,Poly Tull,10000,20000


In [8]:
selected_container

'40ft'

In [10]:
# Dimensions for each container size
dim_20ft = (2.33, 5.9, 2.35)
dim_40ft = (2.33, 11.99, 2.35)
dim_40hq = (2.33, 11.99, 2.68)

# Volume for each container size
cbm_20ft = dim_20ft[0] * dim_20ft[1] * dim_20ft[2]
cbm_40ft = dim_40ft[0] * dim_40ft[1] * dim_40ft[2]
cbm_40hq = dim_40hq[0] * dim_40hq[1] * dim_40hq[2]

# assign CBM for selected container
if selected_container == '20ft':
    cont_cbm = cbm_20ft
elif selected_container == '40ft':
    cont_cbm = cbm_40ft
else:
    cont_cbm = cbm_40hq

In [11]:
df_merged = df_selected.merge(df_box, left_on='item', right_on='Item')
df_merged = df_merged.drop(['Item', 'Width', 'Length', 'Height'], axis=1)
df_merged

,item,demand,capacity,Per Box,Unit Profit,CBM
0,Hi Multi Chiffon K-711,50000,45000,500,0.89,0.199272
1,Satin Taffeta K-1479,40000,100000,250,0.93,0.161721
2,Velvet (320G),4000,100000,150,0.88,0.209250
3,Arcai Slub K-2574,40000,50000,500,0.29,0.180576
4,Poly Tull,10000,20000,500,0.68,0.224532


In [12]:
m = pl.LpProblem('Textiles', pl.LpMaximize)

In [13]:
# Create variables
x = []
for i in df_merged.index:
    x.append(pl.LpVariable(f'q_{i}', cat='Integer') )
    
x = np.array(x)

##### Variables  
$x_i:$ production quantity for item $i$  
$c_i:$ production capacity for item $i$  
$d_i:$ demand for item $i$  
$v_i:$ volume (CBM) of one box for item $i$  
$y_i:$ quantity in yards that will be put in a box for item $i$  
$cv:$ volume (CBM) of selected container  
$p_i:$ unit profit for item $i$  
$N:$ number of items selected

##### Contraints  
* Each item's production must be less than or equal to its capacity: 
\begin{align}
x_i \le c_i  \; \text{ for} \; i = 1,2,...,N
\end{align}  
<br>

* Each item's production must be within $\pm20%$ of its demand:  
\begin{align}
x_i \le d_i - 0.2 \times d_i  
\end{align} 
\begin{align}
x_i \le d_i + 0.2 \times d_i  
\end{align}  
<br>

* Total volume of all items must be less than 0.9 times volume of a container  
  (it's 0.9 because we can't perfectly load boxes without any unused space in a container)
\begin{align}
\sum_{i=1}^{N} \frac{v_i x_i}{y_i} \le 0.9 \times cv
\end{align}  

##### Objective:  
\begin{align}
maximize \sum_{i=1}^{N}p_i x_i 
\end{align} 

In [14]:
# Create constraints

# Production for each item has a capacity constraint
for i in df_merged.index:
    m += (x[i] <= df_merged.capacity[i], f'Production capacity constraint {i}')

# Must produce at least as much as demand
for i in df_merged.index:
    m += (x[i] >= df_merged.demand[i] - 0.2 * df_merged.demand[i], f'Item {i} minimum constraint')
    m += (x[i] <= df_merged.demand[i] + 0.2 * df_merged.demand[i], f'Item {i} maximum constraint')

# Total volume of all boxes must be less than 0.9 times volume of container
m += (pl.lpSum(np.array(df_merged['CBM']) * x / df_merged['Per Box']) <= 0.9 * cont_cbm)

In [15]:
m += pl.lpSum(np.array(df_merged['Unit Profit']) * x)

In [16]:
m

Textiles:
MAXIMIZE
0.89*q_0 + 0.93*q_1 + 0.88*q_2 + 0.29*q_3 + 0.68*q_4 + 0.0
SUBJECT TO
Production_capacity_constraint_0: q_0 <= 45000

Production_capacity_constraint_1: q_1 <= 100000

Production_capacity_constraint_2: q_2 <= 100000

Production_capacity_constraint_3: q_3 <= 50000

Production_capacity_constraint_4: q_4 <= 20000

Item_0_minimum_constraint: q_0 >= 40000

Item_0_maximum_constraint: q_0 <= 60000

Item_1_minimum_constraint: q_1 >= 32000

Item_1_maximum_constraint: q_1 <= 48000

Item_2_minimum_constraint: q_2 >= 3200

Item_2_maximum_constraint: q_2 <= 4800

Item_3_minimum_constraint: q_3 >= 32000

Item_3_maximum_constraint: q_3 <= 48000

Item_4_minimum_constraint: q_4 >= 8000

Item_4_maximum_constraint: q_4 <= 12000

_C1: 0.000398544 q_0 + 0.000646884 q_1 + 0.001395 q_2 + 0.000361152 q_3
 + 0.000449064 q_4 <= 59.0861205

VARIABLES
q_0 free Integer
q_1 free Integer
q_2 free Integer
q_3 free Integer
q_4 free Integer

In [17]:
# Solve the linear optimization problem
result = pl.PULP_CBC_CMD().solve(m)

if result == 1:
    print("Shipment accepted")
else:
    print("Shipment not accepted")

Shipment accepted


In [18]:
# Obtain optimal solution
optimal_soln = m.objective.value()
optimal_soln

88614.88

In [19]:
df_merged['Production'] = [x[i].value() for i in range(len(x))]
df_merged

,item,demand,capacity,Per Box,Unit Profit,CBM,Production
0,Hi Multi Chiffon K-711,50000,45000,500,0.89,0.199272,45000.0
1,Satin Taffeta K-1479,40000,100000,250,0.93,0.161721,32000.0
2,Velvet (320G),4000,100000,150,0.88,0.209250,3200.0
3,Arcai Slub K-2574,40000,50000,500,0.29,0.180576,32000.0
4,Poly Tull,10000,20000,500,0.68,0.224532,9866.0


In [20]:
# Round down to thousands because orders are usually made in thousands and rounding up can cause excess capacity
df_merged['Production'] = ((df_merged['Production'] // 1000) * 1000).astype(int)

In [22]:
df_merged[['item', 'Production']]

,item,Production
0,Hi Multi Chiffon K-711,45000
1,Satin Taffeta K-1479,32000
2,Velvet (320G),3000
3,Arcai Slub K-2574,32000
4,Poly Tull,9000


### 'Production' column shows the optimal quantities we can sell for this customer.